In [2]:
from sklearn.datasets import load_iris
import pandas as pd 
data = load_iris()
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular

In [3]:
y = data.target[1:70]
x = data.data[1:70]

x = pd.DataFrame(x)
x.columns = ['x1', 'x2', 'x3', 'x4']

x['x1'] = np.where(x['x1'] >5, 1, 0)
x['x2'] = np.where(x['x2'] >3, 1, 0)
x['x3'] = np.where(x['x3'] >1, 1, 0)
x['x4'] = np.where(x['x4'] >1, 1, 0)

In [4]:
from sklearn.model_selection import train_test_split

iris_train, VALID, y_train, VALIDy = train_test_split(x, y, random_state=40)

In [5]:
iris_valid, iris_test, y_valid, y_test = train_test_split(VALID, VALIDy, random_state=40)

In [6]:
class_names = ['setosa', 'versicolor']
feature_names = ['x1', 'x2', 'x3', 'x4']
categorical_names = {0 : ['1','0'],
            1 : ['1','0'],
            2 : ['1','0'],
            3 : ['1', '0']}


In [9]:
iris_train = np.array(iris_train)
y_train = np.array(y_train)
iris_valid = np.array(iris_valid)
y_valid = np.array(y_valid)
iris_test = np.array(iris_test)
y_test = np.array(y_test)

In [10]:

explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, x, categorical_names)
explainer.fit(iris_train, y_train, iris_valid, y_valid)

/home/spenser/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(explainer.encoder.transform(iris_train), y_train)
predict_fn = lambda x: c.predict(explainer.encoder.transform(x))


In [12]:
idx = 0
np.random.seed(1)
exp = explainer.explain_instance(iris_test[3], c.predict, threshold=0.95)

In [13]:
# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(iris_test[:, exp.features()] == iris_test[idx][exp.features()], axis=1))[0]
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(iris_train.shape[0])))
print('Anchor test precision: %.2f' % (np.mean(predict_fn(iris_test[fit_anchor]) == predict_fn(iris_test[idx].reshape(1, -1)))))

Anchor: x4 = 1
Anchor test coverage: 0.10
Anchor test precision: 1.00
